#  RAG 체인 구성

---

## RAG란 무엇인가?

### 🎯 핵심 개념
**Retrieval Augmented Generation (RAG)** 는 대규모 언어 모델(LLM)에 외부 지식을 연결하여 더 정확하고 최신의 정보를 제공하는 AI 프레임워크입니다.

### 🔍 RAG의 작동 원리
```
사용자 질문 → 관련 문서 검색 → 컨텍스트와 함께 LLM에 전달 → 답변 생성
```

### 📊 RAG vs 일반 LLM 비교
| 구분 | 일반 LLM | RAG |
|------|----------|-----|
| 정보 소스 | 사전 훈련 데이터만 | 외부 지식베이스 + 사전 훈련 데이터 |
| 최신성 | 훈련 시점까지 | 실시간 업데이트 가능 |
| 정확성 | 환각(hallucination) 가능성 | 검증된 문서 기반 답변 |
| 사용 사례 | 일반적인 질문 답변 | 특정 도메인의 전문적 답변 |

---

## 환경 설정

### 🛠️ 필수 라이브러리 설치

```bash
# 기본 LangChain 패키지
pip install langchain langchain-community 

# 임베딩 모델
pip install langchain-openai langchain-huggingface

# 벡터 저장소
pip install langchain-chroma

# 문서 처리
pip install pypdf 

# 웹 스크래핑
pip install beautifulsoup4

# 토크나이저
pip install tiktoken transformers sentence-transformers

# 실험적 기능 (SemanticChunker)
pip install langchain-experimental


uv add langchain_community
uv add pypdf  
uv add bs4  

### 🔑 환경 변수 설정

In [1]:
# .env 파일 생성
from dotenv import load_dotenv
import os

load_dotenv()

# OpenAI API 키 설정 (필요시)
# OPENAI_API_KEY=your_openai_api_key_here

True

### 📋 기본 라이브러리 import


In [2]:
import os
from glob import glob
from pprint import pprint
import json
from pathlib import Path

---

## 문서 로더 (Document Loaders)

### 🎯 문서 로더란?
**Document Loader**는 다양한 소스에서 문서를 로드하여 LangChain의 `Document` 객체로 변환하는 도구입니다.

### 📄 Document 객체 구조
```python
from langchain_core.documents import Document

# Document 객체의 기본 구조
document = Document(
    page_content="문서의 텍스트 내용",
    metadata={
        "source": "문서 출처",
        "page": 1,
        "title": "문서 제목"
    }
)
```

### 📄 문서 로더의 종류
- PDF 파일 로더
- 웹 페이지 로더 
- CSV 데이터 로더
- 디렉토리 로더
- HTML 데이터 로더
- JSON 데이터 로더
- Markdown 데이터 로더
- Microsoft Office 데이터 로더


### 1. 🌐 웹 문서 로더 (WebBaseLoader)

In [3]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

# 기본 웹 문서 로드
web_loader = WebBaseLoader(
    web_paths=[
        "https://python.langchain.com/docs/tutorials/rag/",
        "https://js.langchain.com/docs/tutorials/rag/"
    ]
)

# 문서 로드
web_docs = web_loader.load()
print(f"로드된 문서 수: {len(web_docs)}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


로드된 문서 수: 2


In [4]:
print(f"첫 번째 문서 내용:\n{web_docs[0].page_content[:1500]}...")  # 앞부분만 출력

첫 번째 문서 내용:





Build a Retrieval Augmented Generation (RAG) App: Part 1 | 🦜️🔗 LangChain








Skip to main contentOur new LangChain Academy Course Deep Research with LangGraph is now live! Enroll for free.IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use example selectorsHow to add a semantic layer over graph databaseHow to invoke runnables in parallelHow to 

In [5]:
print(f"첫 번째 문서 메타데이터: {web_docs[0].metadata}")

첫 번째 문서 메타데이터: {'source': 'https://python.langchain.com/docs/tutorials/rag/', 'title': 'Build a Retrieval Augmented Generation (RAG) App: Part 1 | 🦜️🔗 LangChain', 'description': 'One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or RAG.', 'language': 'en'}


### 2. 📊 CSV 파일 로더 (CSVLoader)

In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# 기본 CSV 로드
csv_loader = CSVLoader("./data/kbo_teams_2023.csv", encoding="utf-8")
csv_docs = csv_loader.load()

print(f"문서 수: {len(csv_docs)}")

문서 수: 10


In [7]:
print(f"첫 번째 문서 내용:\n{csv_docs[0].page_content}")

첫 번째 문서 내용:
Team: KIA 타이거즈
City: 광주
Founded: 1982
Home Stadium: 광주-기아 챔피언스 필드
Championships: 11
Introduction: KBO 리그의 전통 강호로, 역대 최다 우승 기록을 보유하고 있다. '타이거즈 스피릿'으로 유명하며, 양현종, 안치홍 등 스타 선수들을 배출했다. 광주를 연고로 하는 유일한 프로야구팀으로 지역 사랑이 강하다.


In [8]:
print(f"첫 번째 문서 메타데이터: {csv_docs[0].metadata}")

첫 번째 문서 메타데이터: {'source': './data/kbo_teams_2023.csv', 'row': 0}


In [9]:
# 소스 컬럼 지정 및 인코딩 설정
csv_loader_advanced = CSVLoader(
    file_path="./data/kbo_teams_2023.csv",
    source_column="Team",      # 이 컬럼이 메타데이터의 source가 됨
    content_columns=["Team", "Introduction"],  # 이 컬럼이 문서의 내용이 됨
    metadata_columns=["Founded", "City"],  # 이 컬럼이 메타데이터에 추가됨
    encoding="utf-8",          # 인코딩 명시
    csv_args={
        "delimiter": ",",      # 구분자
        "quotechar": '"',      # 인용 문자
    }
)

csv_docs_advanced = csv_loader_advanced.load()

# 문서 수와 첫 번째 문서 내용 출력
print(f"문서 수: {len(csv_docs_advanced)}")

문서 수: 10


In [10]:
print(f"마지막 문서 내용:\n{csv_docs_advanced[-1].page_content}")

마지막 문서 내용:
Team: 한화 이글스
Introduction: 대전을 연고로 하는 구단으로, 1999년 한국시리즈 우승을 차지했다. '절친 야구'로 유명한 정근우, 이용규 등의 선수들이 팀을 대표했다. 최근 몇 년간 성적이 부진했지만, 젊은 선수들의 육성에 힘쓰며 재건을 꾀하고 있다.


In [11]:
print(f"마지막 문서 메타데이터: {csv_docs_advanced[-1].metadata}")

마지막 문서 메타데이터: {'source': '한화 이글스', 'row': 9, 'Founded': '1986', 'City': '대전'}


### 3. 📖 PDF 파일 로더 

- **PyPDFLoader**

In [12]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/labor_law.pdf')

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

PDF 문서 개수: 20


In [13]:
# 각 페이지별 정보 확인
for i, doc in enumerate(pdf_docs[:3]):
    print(f"페이지 {i+1}: {len(doc.page_content)} 문자")
    print(f"메타데이터: {doc.metadata}")

페이지 1: 1811 문자
메타데이터: {'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2024-10-15T14:45:34+09:00', 'moddate': '2024-10-15T14:45:34+09:00', 'source': './data/labor_law.pdf', 'total_pages': 20, 'page': 0, 'page_label': '1'}
페이지 2: 1709 문자
메타데이터: {'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2024-10-15T14:45:34+09:00', 'moddate': '2024-10-15T14:45:34+09:00', 'source': './data/labor_law.pdf', 'total_pages': 20, 'page': 1, 'page_label': '2'}
페이지 3: 2164 문자
메타데이터: {'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2024-10-15T14:45:34+09:00', 'moddate': '2024-10-15T14:45:34+09:00', 'source': './data/labor_law.pdf', 'total_pages': 20, 'page': 2, 'page_label': '3'}


In [14]:
print(pdf_docs[0].page_content[:1000])  # 첫 페이지의 내용 일부 출력

법제처                                                            1                                                       국가법령정보센터
근로기준법
 
근로기준법
[시행 2021. 11. 19.] [법률 제18176호, 2021. 5. 18., 일부개정]
고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534
고용노동부 (근로기준정책과 - 소년) 044-202-7535
고용노동부 (근로기준정책과 - 임금) 044-202-7548
고용노동부 (여성고용정책과 - 여성) 044-202-7475
고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545
고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973
고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530
고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549
       제1장 총칙
 
제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는
국민경제의 발전을 꾀하는 것을 목적으로 한다.
 
제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.>
1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.
2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다.
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체
결된 계약을 말한다.
5. “임금”이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품

- **다른 PDF 로더들**

In [15]:
from langchain_community.document_loaders import (
    PyMuPDFLoader,
    PDFMinerLoader
)

In [16]:
# PyMuPDF 로더 (빠른 처리)
pymupdf_loader = PyMuPDFLoader("./data/labor_law.pdf")
pdf_docs = pymupdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

# 각 페이지별 정보 확인
for i, doc in enumerate(pdf_docs[:3]):
    print(f"페이지 {i+1}: {len(doc.page_content)} 문자")
    print(f"메타데이터: {doc.metadata}")

PDF 문서 개수: 20
페이지 1: 1811 문자
메타데이터: {'producer': 'iText 2.1.7 by 1T3XT', 'creator': '', 'creationdate': '2024-10-15T14:45:34+09:00', 'source': './data/labor_law.pdf', 'file_path': './data/labor_law.pdf', 'total_pages': 20, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-15T14:45:34+09:00', 'trapped': '', 'modDate': "D:20241015144534+09'00'", 'creationDate': "D:20241015144534+09'00'", 'page': 0}
페이지 2: 1709 문자
메타데이터: {'producer': 'iText 2.1.7 by 1T3XT', 'creator': '', 'creationdate': '2024-10-15T14:45:34+09:00', 'source': './data/labor_law.pdf', 'file_path': './data/labor_law.pdf', 'total_pages': 20, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-15T14:45:34+09:00', 'trapped': '', 'modDate': "D:20241015144534+09'00'", 'creationDate': "D:20241015144534+09'00'", 'page': 1}
페이지 3: 2164 문자
메타데이터: {'producer': 'iText 2.1.7 by 1T3XT', 'creator': '', 'creationdate': '2024-10-15T14:45:34+09:00

In [17]:
print(pdf_docs[0].page_content[:1000])  # 첫 페이지의 내용 일부 출력

법제처                                                            1                                                       국가법령정보센터
근로기준법
 
근로기준법
[시행 2021. 11. 19.] [법률 제18176호, 2021. 5. 18., 일부개정]
고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534
고용노동부 (근로기준정책과 - 소년) 044-202-7535
고용노동부 (근로기준정책과 - 임금) 044-202-7548
고용노동부 (여성고용정책과 - 여성) 044-202-7475
고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545
고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973
고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530
고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549
       제1장 총칙
 
제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는
국민경제의 발전을 꾀하는 것을 목적으로 한다.
 
제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.>
1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.
2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다.
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체
결된 계약을 말한다.
5. “임금”이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품

In [18]:
# PDFMiner 로더 (정확한 텍스트 추출)
pdfminer_loader = PDFMinerLoader("./data/labor_law.pdf")
pdf_docs = pdfminer_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

# 각 페이지별 정보 확인
for i, doc in enumerate(pdf_docs[:3]):
    print(f"페이지 {i+1}: {len(doc.page_content)} 문자")
    print(f"메타데이터: {doc.metadata}")

PDF 문서 개수: 1
페이지 1: 40085 문자
메타데이터: {'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PDFMiner', 'creationdate': '2024-10-15T14:45:34+09:00', 'moddate': '2024-10-15T14:45:34+09:00', 'total_pages': 20, 'source': './data/labor_law.pdf'}


In [19]:
print(pdf_docs[0].page_content[:1000])  # 첫 페이지의 내용 일부 출력

근로기준법

근로기준법

[시행 2021. 11. 19.] [법률 제18176호, 2021. 5. 18., 일부개정]

고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534

고용노동부 (근로기준정책과 - 소년) 044-202-7535

고용노동부 (근로기준정책과 - 임금) 044-202-7548

고용노동부 (여성고용정책과 - 여성) 044-202-7475

고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545

고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973

고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530

고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549

       제1장 총칙

제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는

국민경제의 발전을 꾀하는 것을 목적으로 한다.

제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.>

1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.

2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를

말한다.

3. “근로”란 정신노동과 육체노동을 말한다.

4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체

결된 계약을 말한다.

5. “임금”이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품을

말한다.

6. “평균임금”이란 이를 산정하여야 할 사유가 발생한 날 이전 3개월 동안에 그 근로자에게 지급된 임금의 총액을

그 기간의 총일수로 나눈 금액을 말한다. 근로자가 취업한 후 3개월 미

### 4. 📝 텍스트 파일 로더 (TextLoader)

In [20]:
from langchain_community.document_loaders import TextLoader

# 단일 텍스트 파일 로드
text_loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
text_docs = text_loader.load()

print(f"문서 수: {len(text_docs)}")
print(f"첫 번째 문서 내용:\n{text_docs[0].page_content[:1000]}")  # 첫 1000자 출력
print(f"첫 번째 문서 메타데이터: {text_docs[0].metadata}")

문서 수: 1
첫 번째 문서 내용:
1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.

2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조또에 고급 블랙 트러플을 듬뿍 얹어 풍부한 향과 맛을 즐길 수 있는 메뉴입니다. 24개월 숙성된 파르미지아노 레지아노 치즈를 사용하여 깊은 맛을 더했으며, 주문 즉시 조리하여 최상의 상태로 제공됩니다.

3. 연어 타르타르
   • 가격: ₩18,000
   • 주요 식재료: 노르웨이산 생연어, 아보카도, 케이퍼, 적양파
   • 설명: 신선한 노르웨이산 생연어를 곱게 다져 아보카도, 케이퍼, 적양파와 함께 섞어 만든 타르타르입니다. 레몬 드레싱으로 상큼한 맛을 더했으며, 바삭한 브리오쉬 토스트와 함께 제공됩니다. 전채요리로 완벽한 메뉴입니다.

4. 버섯 크림 수프
   • 가격: ₩10,000
   • 주요 식재료: 양송이버섯, 표고버섯, 생크림, 트러플 오일
   • 설명: 양송이버섯과 표고버섯을 오랜 시간 정성스레 끓여 만든 크림 수프입니다. 부드러운 텍스처와 깊은 버섯 향이 특징이며, 최상급 트러플 오일을 살짝 뿌려 고급스러운 향을 더했습니다. 파슬리를 곱게 다져 고명으로 올려 제공됩니다.

5. 가든 샐러드
   • 가격: ₩12,000
   • 주요 식재료: 유기농 믹스 그린, 체리 토마토, 오이, 당근, 발사믹 드레싱
   • 설명: 신선한 유기농 채소들로 구성된 건강한 샐러드입니다. 아삭한 식감의 믹스 그린에 달콤한 체리 토마토, 오이, 당

In [21]:
# 디렉토리 내 모든 텍스트 파일 로드
from langchain_community.document_loaders import DirectoryLoader

directory_loader = DirectoryLoader(
    "./data/",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)
all_text_docs = directory_loader.load()

print(f"전체 문서 수: {len(all_text_docs)}")
print(f"첫 번째 문서 내용:\n{all_text_docs[0].page_content[:1000]}")  # 첫 1000자 출력
print(f"첫 번째 문서 메타데이터: {all_text_docs[0].metadata}")

전체 문서 수: 2
첫 번째 문서 내용:
1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.

2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조또에 고급 블랙 트러플을 듬뿍 얹어 풍부한 향과 맛을 즐길 수 있는 메뉴입니다. 24개월 숙성된 파르미지아노 레지아노 치즈를 사용하여 깊은 맛을 더했으며, 주문 즉시 조리하여 최상의 상태로 제공됩니다.

3. 연어 타르타르
   • 가격: ₩18,000
   • 주요 식재료: 노르웨이산 생연어, 아보카도, 케이퍼, 적양파
   • 설명: 신선한 노르웨이산 생연어를 곱게 다져 아보카도, 케이퍼, 적양파와 함께 섞어 만든 타르타르입니다. 레몬 드레싱으로 상큼한 맛을 더했으며, 바삭한 브리오쉬 토스트와 함께 제공됩니다. 전채요리로 완벽한 메뉴입니다.

4. 버섯 크림 수프
   • 가격: ₩10,000
   • 주요 식재료: 양송이버섯, 표고버섯, 생크림, 트러플 오일
   • 설명: 양송이버섯과 표고버섯을 오랜 시간 정성스레 끓여 만든 크림 수프입니다. 부드러운 텍스처와 깊은 버섯 향이 특징이며, 최상급 트러플 오일을 살짝 뿌려 고급스러운 향을 더했습니다. 파슬리를 곱게 다져 고명으로 올려 제공됩니다.

5. 가든 샐러드
   • 가격: ₩12,000
   • 주요 식재료: 유기농 믹스 그린, 체리 토마토, 오이, 당근, 발사믹 드레싱
   • 설명: 신선한 유기농 채소들로 구성된 건강한 샐러드입니다. 아삭한 식감의 믹스 그린에 달콤한 체리 토마토, 오이

### 🎯 실습 1: 웹 문서 로더 연습

In [22]:
# 다음 웹 페이지들을 로드하고 메타데이터를 출력해보세요
urls = [
    "https://python.langchain.com/docs/tutorials/",
    "https://python.langchain.com/docs/concepts/"
]


# 여기에 코드를 작성하세요

# 기본 웹 문서 로드
web_loader = WebBaseLoader(
    web_paths=urls
)

# 문서 로드
web_docs = web_loader.load()


for i, doc in enumerate(web_docs[:2]):
    print(f"페이지 {i+1}: {len(doc.page_content)} 문자")
    print(f"메타데이터: {doc.metadata}")

페이지 1: 9859 문자
메타데이터: {'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | 🦜️🔗 LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'language': 'en'}
페이지 2: 16323 문자
메타데이터: {'source': 'https://python.langchain.com/docs/concepts/', 'title': 'Conceptual guide | 🦜️🔗 LangChain', 'description': 'This guide provides explanations of the key concepts behind the LangChain framework and AI applications more broadly.', 'language': 'en'}


---

## 텍스트 분할 (Text Splitting)

### 🎯 텍스트 분할이 필요한 이유
1. **토큰 제한**: LLM은 입력 토큰 수에 제한이 있음
2. **검색 정확도**: 작은 청크가 더 정확한 검색 결과 제공
3. **메모리 효율성**: 대용량 문서의 효율적 처리

### 📊 분할 전략 비교
| 방법 | 장점 | 단점 | 사용 사례 |
|------|------|------|----------|
| CharacterTextSplitter | 단순, 빠름 | 문맥 고려 안함 | 간단한 텍스트 |
| RecursiveCharacterTextSplitter | 문맥 보존 우수 | 계산 복잡 | 일반적인 문서 |
| SemanticChunker | 의미 기반 분할 | 느림, 비용 많음 | 중요한 문서 |
| TokenTextSplitter | 정확한 토큰 수 | 토크나이저 의존 | API 비용 최적화 |

In [23]:
# PDF 로더 초기화
pdf_loader = PyPDFLoader('./data/labor_law.pdf', mode='single')  # 'single' 또는 'page' 모드 선택 가능

# 동기 로딩
pdf_docs = pdf_loader.load()
print(f'PDF 문서 개수: {len(pdf_docs)}')

PDF 문서 개수: 1


### 1. CharacterTextSplitter

In [24]:
long_text = pdf_docs[0].page_content
print(f'첫 번째 문서의 텍스트 길이: {len(long_text)}')

첫 번째 문서의 텍스트 길이: 39539


In [25]:
from langchain_text_splitters import CharacterTextSplitter

# 기본 설정
text_splitter = CharacterTextSplitter(
    separator="\n\n",        # 구분자
    chunk_size=1000,         # 청크 크기
    chunk_overlap=200,       # 중복 크기
    length_function=len,     # 길이 측정 함수
    is_separator_regex=False # 정규식 여부
)

# 텍스트 분할
chunks = text_splitter.split_text(long_text)
print(f"분할된 청크 수: {len(chunks)}")

분할된 청크 수: 1


In [26]:
from langchain_text_splitters import CharacterTextSplitter

# 기본 설정
text_splitter = CharacterTextSplitter(
    separator="\n",        # 구분자
    chunk_size=1000,         # 청크 크기
    chunk_overlap=200,       # 중복 크기
    length_function=len,     # 길이 측정 함수
    is_separator_regex=False # 정규식 여부
)

# 텍스트 분할
chunks = text_splitter.split_text(long_text)
print(f"분할된 청크 수: {len(chunks)}")

for i, chunk in enumerate(chunks[:10]):
    print(f"청크 {i+1} 길이: {len(chunk)} 문자")    

분할된 청크 수: 50
청크 1 길이: 936 문자
청크 2 길이: 944 문자
청크 3 길이: 971 문자
청크 4 길이: 930 문자
청크 5 길이: 973 문자
청크 6 길이: 961 문자
청크 7 길이: 995 문자
청크 8 길이: 971 문자
청크 9 길이: 996 문자
청크 10 길이: 972 문자


In [27]:
import re
from langchain.text_splitter import CharacterTextSplitter


# 법률 문서와 공문서에 적합한 문장 분할 정규식 패턴
# 문장 끝 후 공백이 있고, 특정 패턴이 따라오지 않는 경우만 분할 (목록 번호나 조항 번호 앞, 괄호로 된 항목 표시 앞, 기타 구두점 앞)
sentence_pattern = r'(?<=[.!?])\s+(?!\s*(?:\d+|호|조|항|]|\)|[가-힣]{1,2}\s*\)|[A-Za-z]\s*\)|[,;:]))'

# 정규식을 사용한 문장 단위 분할기 생성
sentence_splitter = CharacterTextSplitter(
    separator=sentence_pattern,
    chunk_size=500,
    chunk_overlap=100,
    is_separator_regex=True,
    keep_separator=True
)

# 문장 단위로 분할
sentence_chunks = sentence_splitter.split_text(long_text)
print(f"문장 단위로 분할된 청크 수: {len(sentence_chunks)}")
for i, chunk in enumerate(sentence_chunks[:10]):
    print(f"청크 {i+1} 길이: {len(chunk)} 문자")

Created a chunk of size 620, which is longer than the specified 500
Created a chunk of size 555, which is longer than the specified 500
Created a chunk of size 539, which is longer than the specified 500
Created a chunk of size 548, which is longer than the specified 500


문장 단위로 분할된 청크 수: 103
청크 1 길이: 620 문자
청크 2 길이: 470 문자
청크 3 길이: 465 문자
청크 4 길이: 410 문자
청크 5 길이: 492 문자
청크 6 길이: 490 문자
청크 7 길이: 423 문자
청크 8 길이: 395 문자
청크 9 길이: 350 문자
청크 10 길이: 285 문자


In [28]:
print(f"첫 번째 문서 청크 내용: {sentence_chunks[0]}")

첫 번째 문서 청크 내용: 법제처                                                            1                                                       국가법령정보센터
근로기준법
 
근로기준법
[시행 2021. 11. 19.] [법률 제18176호, 2021. 5. 18., 일부개정]
고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534
고용노동부 (근로기준정책과 - 소년) 044-202-7535
고용노동부 (근로기준정책과 - 임금) 044-202-7548
고용노동부 (여성고용정책과 - 여성) 044-202-7475
고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545
고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973
고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530
고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549
       제1장 총칙
 
제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는
국민경제의 발전을 꾀하는 것을 목적으로 한다.


In [29]:
print(f"두 번째 문서 청크 내용: {sentence_chunks[1]}")

두 번째 문서 청크 내용: 제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.>
1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.
2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다.
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체
결된 계약을 말한다.
5. “임금”이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품을
말한다.
6. “평균임금”이란 이를 산정하여야 할 사유가 발생한 날 이전 3개월 동안에 그 근로자에게 지급된 임금의 총액을
그 기간의 총일수로 나눈 금액을 말한다.


In [30]:
print(f"세 번째 문서 청크 내용: {sentence_chunks[2]}")

세 번째 문서 청크 내용: “평균임금”이란 이를 산정하여야 할 사유가 발생한 날 이전 3개월 동안에 그 근로자에게 지급된 임금의 총액을
그 기간의 총일수로 나눈 금액을 말한다. 근로자가 취업한 후 3개월 미만인 경우도 이에 준한다.
7. “1주”란 휴일을 포함한 7일을 말한다.
8. “소정(所定)근로시간”이란 제50조, 제69조 본문 또는 「산업안전보건법」 제139조제1항에 따른 근로시간의 범위에
서 근로자와 사용자 사이에 정한 근로시간을 말한다.
9. “단시간근로자”란 1주 동안의 소정근로시간이 그 사업장에서 같은 종류의 업무에 종사하는 통상 근로자의 1주
동안의 소정근로시간에 비하여 짧은 근로자를 말한다.
② 제1항제6호에 따라 산출된 금액이 그 근로자의 통상임금보다 적으면 그 통상임금액을 평균임금으로 한다.
 
제3조(근로조건의 기준) 이 법에서 정하는 근로조건은 최저기준이므로 근로 관계 당사자는 이 기준을 이유로 근로조건
을 낮출 수 없다.


### 2. RecursiveCharacterTextSplitter

- 재귀적으로 텍스트를 분할하여 문맥을 최대한 보존하는 분할 도구


In [31]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 기본 재귀 분할기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", sentence_pattern]  # 우선순위 순서 (큰 구분자부터 작은 구분자 순서로 재귀적 분할)
    # separators=["\n\n", "\n", " ", ""]  # 우선순위 순서 (큰 구분자부터 작은 구분자 순서로 재귀적 분할)
)

# Document 객체 분할
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 청크 수: {len(chunks)}")

# 각 청크의 길이 확인
for i, chunk in enumerate(chunks[:3]):
    print(f"청크 {i+1}: {len(chunk.page_content)} 문자")

생성된 청크 수: 50
청크 1: 936 문자
청크 2: 944 문자
청크 3: 971 문자


### 3. 토큰 기반 분할

#### 🔧 TikToken 토크나이저 기반 분할
- OpenAI 임베딩 모델이 사용하는 토크나이저를 사용하여 정확한 토큰 수로 텍스트를 분할하는 도구

In [32]:
# OpenAI 토크나이저 사용
token_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",  # GPT-4 인코딩
    chunk_size=500,              # 토큰 수 기준
    chunk_overlap=100
)

chunks = token_splitter.split_documents([pdf_docs[0]])

# 토큰 수 확인
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")

for i, chunk in enumerate(chunks[:3]):
    token_count = len(tokenizer.encode(chunk.page_content))
    print(f"청크 {i+1}: {token_count} 토큰, {len(chunk.page_content)} 문자")

청크 1: 492 토큰, 620 문자
청크 2: 449 토큰, 476 문자
청크 3: 475 토큰, 473 문자


#### 🤗 Hugging Face 토크나이저

In [33]:
from transformers import AutoTokenizer

# BGE-M3 토크나이저 사용
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

hf_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=300,
    chunk_overlap=50
)

chunks = hf_splitter.split_documents([pdf_docs[0]])

# 토큰 수 확인
for i, chunk in enumerate(chunks[:3]):
    token_count = len(tokenizer(chunk.page_content)["input_ids"])
    print(f"청크 {i+1}: {token_count} 토큰, {len(chunk.page_content)} 문자")

c:\Users\ys\ktds-llm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


청크 1: 284 토큰, 620 문자
청크 2: 298 토큰, 594 문자
청크 3: 273 토큰, 464 문자


### 4. **Semantic Chunking**

- **SemanticChunker**는 텍스트를 의미 단위로 **분할**하는 특수한 텍스트 분할도구 

- 단순 길이 기반이 아닌 **의미 기반**으로 텍스트를 청크화하는데 효과적

- **breakpoint_threshold_type**: Text Splitting의 다양한 임계값(Threshold) 설정 방식 (통계적 기법) 

    - **Gradient** 방식: 임베딩 벡터 간의 **기울기 변화**를 기준으로 텍스트를 분할
    - **Percentile** 방식: 임베딩 거리의 **백분위수**를 기준으로 분할 지점을 결정 (기본값: 95%)
    - **Standard Deviation** 방식: 임베딩 거리의 **표준편차**를 활용하여 유의미한 변화점을 찾아서 분할
    - **Interquartile** 방식: 임베딩 거리의 **사분위수 범위**를 기준으로 이상치를 감지하여 분할

- 설치: pip install langchain_experimental 또는 uv add langchain_experimental


In [34]:
from langchain_experimental.text_splitter import SemanticChunker 
from langchain_openai.embeddings import OpenAIEmbeddings

# 임베딩 모델을 사용하여 SemanticChunker를 초기화 
text_splitter = SemanticChunker(
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),         # OpenAI 임베딩 사용
    breakpoint_threshold_type="gradient",  # 임계값 타입 설정 (gradient, percentile, standard_deviation, interquartile)
)

In [35]:
chunks = text_splitter.split_documents(pdf_docs)

print(f"생성된 청크 수: {len(chunks)}")
print(f"각 청크의 길이: {list(len(chunk.page_content) for chunk in chunks)}")

생성된 청크 수: 40
각 청크의 길이: [151, 542, 3166, 5231, 129, 50, 911, 2377, 313, 137, 1999, 2, 275, 2412, 9, 1209, 111, 5859, 2120, 351, 223, 514, 2, 179, 1996, 2, 515, 2, 391, 2, 97, 1845, 2512, 1151, 175, 2, 1565, 82, 426, 295]


### 🎯 실습 2: 텍스트 분할 비교

In [36]:
# 다음 텍스트를 다양한 방법으로 분할하고 결과를 비교해보세요
sample_text = """
제1번 우리나라는 대한민국입니다.한화이글스는 대전을 연고로 하는 구단으로, 1999년 한국시리즈 우승을 차지했다. '절친 야구'로 유명한 정근우, 이용규 등의 선수들이 팀을 대표했다. 최근 몇 년간 성적이 부진했지만,  젊은 선수들의 육성에 힘쓰며 재건을 꾀하고 있다.



제2번 인공지능은 현대 기술의 핵심입니다. 
머신러닝을 통해 컴퓨터는 학습할 수 있습니다.


제3번 딥러닝은 신경망을 기반으로 합니다.
자연어 처리는 텍스트를 이해하는 기술입니다.


제4번 컴퓨터 비전은 이미지를 분석합니다.
강화학습은 행동을 통해 학습합니다.

1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.
2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다.
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체
결된 계약을 말한다.
5. “임금”이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품을
말한다.
6. “평균임금”이란 이를 산정하여야 할 사유가 발생한 날 이전 3개월 동안에 그 근로자에게 지급된 임금의 총액을
그 기간의 총일수로 나눈 금액을 말한다.


2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조또에 고급 블랙 트러플을 듬뿍 얹어 풍부한 향과 맛을 즐길 수 있는 메뉴입니다. 24개월 숙성된 파르미지아노 레지아노 치즈를 사용하여 깊은 맛을 더했으며, 주문 즉시 조리하여 최상의 상태로 제공됩니다.

3. 연어 타르타르
   • 가격: ₩18,000
   • 주요 식재료: 노르웨이산 생연어, 아보카도, 케이퍼, 적양파
   • 설명: 신선한 노르웨이산 생연어를 곱게 다져 아보카도, 케이퍼, 적양파와 함께 섞어 만든 타르타르입니다. 레몬 드레싱으로 상큼한 맛을 더했으며, 바삭한 브리오쉬 토스트와 함께 제공됩니다. 전채요리로 완벽한 메뉴입니다.

   

"""

# 여기에 코드를 작성하세요
print("##########################################################################")

# 1. 문자 단위 분할기
text_splitter = CharacterTextSplitter(
    separator="\n",        # 구분자
    chunk_size=1000,         # 청크 크기
    chunk_overlap=200,       # 중복 크기
    length_function=len,     # 길이 측정 함수
    is_separator_regex=False # 정규식 여부
)

# 텍스트 분할
chunks = text_splitter.split_text(sample_text)
print(f"문자 단위 분할기 분할된 청크 수: {len(chunks)}")

for i, chunk in enumerate(chunks[:10]):
    print(f"문자 단위 분할기 청크 {i+1} 길이: {len(chunk)} 문자")    

print("##########################################################################")

# 2. 재귀 분할기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", sentence_pattern]  # 우선순위 순서 (큰 구분자부터 작은 구분자 순서로 재귀적 분할)
    # separators=["\n\n", "\n", " ", ""]  # 우선순위 순서 (큰 구분자부터 작은 구분자 순서로 재귀적 분할)
)

# Document 객체 분할
chunks = text_splitter.split_text(sample_text)
print(f"재귀 분할기 생성된 청크 수: {len(chunks)}")

# 각 청크의 길이 확인
for i, chunk in enumerate(chunks[:3]):
    print(f"재귀 분할기 청크 {i+1}: {len(chunk)} 문자")
    
print("##########################################################################")


# 3. OpenAI 토크나이저 사용
token_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",  # GPT-4 인코딩
    chunk_size=500,              # 토큰 수 기준
    chunk_overlap=100
)

chunks = token_splitter.split_text(sample_text)

# 토큰 수 확인
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")

for i, chunk in enumerate(chunks[:3]):
    token_count = len(tokenizer.encode(chunk))
    print(f"OpenAI 토크나이저 청크 {i+1}: {token_count} 토큰, {len(chunk)} 문자")


print("##########################################################################")


# BGE-M3 토크나이저 사용
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

hf_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=300,
    chunk_overlap=50
)

chunks = hf_splitter.split_text(sample_text)

# 토큰 수 확인
for i, chunk in enumerate(chunks[:3]):
    token_count = len(tokenizer(chunk)["input_ids"])
    print(f"BGE-M3 토크나이저 청크 {i+1}: {token_count} 토큰, {len(chunk)} 문자")


print("##########################################################################")


# 임베딩 모델을 사용하여 SemanticChunker를 초기화 
text_splitter = SemanticChunker(
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"),         # OpenAI 임베딩 사용
    breakpoint_threshold_type="gradient",  # 임계값 타입 설정 (gradient, percentile, standard_deviation, interquartile)
)
chunks = text_splitter.split_text(sample_text)

print(f"SemanticChunker 생성된 청크 수: {len(chunks)}")
print(f"SemanticChunker 각 청크의 길이: {list(len(chunk) for chunk in chunks)}")

##########################################################################
문자 단위 분할기 분할된 청크 수: 2
문자 단위 분할기 청크 1 길이: 963 문자
문자 단위 분할기 청크 2 길이: 324 문자
##########################################################################
재귀 분할기 생성된 청크 수: 2
재귀 분할기 청크 1: 906 문자
재귀 분할기 청크 2: 193 문자
##########################################################################
OpenAI 토크나이저 청크 1: 284 토큰, 301 문자
OpenAI 토크나이저 청크 2: 463 토큰, 484 문자
OpenAI 토크나이저 청크 3: 437 토큰, 407 문자
##########################################################################
BGE-M3 토크나이저 청크 1: 155 토큰, 301 문자
BGE-M3 토크나이저 청크 2: 245 토큰, 484 문자
BGE-M3 토크나이저 청크 3: 248 토큰, 407 문자
##########################################################################
SemanticChunker 생성된 청크 수: 3
SemanticChunker 각 청크의 길이: [64, 209, 816]


---

## 문서 임베딩 (Document Embedding)

### 🎯 임베딩이란?
텍스트를 고차원 벡터 공간의 숫자 배열로 변환하여 의미적 유사도를 계산할 수 있게 하는 기술입니다.

### 📊 임베딩 모델 비교
| 모델 | 차원 | 언어 지원 | 비용 | 성능 | 사용 사례 |
|------|------|----------|------|------|----------|
| OpenAI text-embedding-3-small | 1536 | 다국어 | 유료 | 높음 | 프로덕션 |
| OpenAI text-embedding-3-large | 3072 | 다국어 | 유료 | 최고 | 고성능 요구 |
| BAAI/bge-m3 | 1024 | 다국어 | 무료 | 높음 | 한국어 특화 |
| sentence-transformers/all-MiniLM-L6-v2 | 384 | 영어 | 무료 | 중간 | 로컬 개발 |

### 1. OpenAI 임베딩

#### 🔧 기본 설정

In [37]:
from langchain_openai import OpenAIEmbeddings

# 기본 임베딩 모델
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1536,           # 차원 수 (기본값)
    show_progress_bar=True,    # 진행률 표시
    max_retries=3             # 재시도 횟수
)

print(f"임베딩 차원: {embeddings_model.dimensions}")
print(f"컨텍스트 길이: {embeddings_model.embedding_ctx_length}")

임베딩 차원: 1536
컨텍스트 길이: 8191


#### 📝 문서 임베딩

In [38]:
# 문서 컬렉션 임베딩
documents = [
    "인공지능은 컴퓨터 과학의 한 분야입니다.",
    "머신러닝은 인공지능의 하위 분야입니다.",
    "딥러닝은 머신러닝의 한 종류입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하는 기술입니다.",
    "컴퓨터 비전은 이미지를 분석하는 기술입니다."
]

# 배치 임베딩 (효율적)
doc_embeddings = embeddings_model.embed_documents(documents)
print(f"임베딩 벡터 수: {len(doc_embeddings)}")
print(f"각 벡터 차원: {len(doc_embeddings[0])}")

100%|██████████| 1/1 [00:00<00:00,  1.21it/s]

임베딩 벡터 수: 5
각 벡터 차원: 1536


In [39]:
# 쿼리 임베딩
query = "AI 기술에 대해 알려주세요"
query_embedding = embeddings_model.embed_query(query)
print(f"쿼리 임베딩 차원: {len(query_embedding)}")

100%|██████████| 1/1 [00:00<00:00,  2.66it/s]

쿼리 임베딩 차원: 1536


#### 💡 차원 축소 활용

In [40]:
# 비용 절약을 위한 차원 축소
compact_embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
    dimensions=512  # 원래 1536에서 512로 축소
)

# 성능과 비용의 균형점 찾는 것이 중요!!!
compact_doc_embeddings = compact_embeddings.embed_documents(documents)
print(f"축소된 임베딩 벡터 수: {len(compact_doc_embeddings)}")
print(f"축소된 각 벡터 차원: {len(compact_doc_embeddings[0])}") 

축소된 임베딩 벡터 수: 5
축소된 각 벡터 차원: 512


### 2. Hugging Face 임베딩

#### 🤗 BGE-M3 모델 (한국어 우수)


In [41]:
from langchain_huggingface import HuggingFaceEmbeddings

# BGE-M3 모델 (다국어, 한국어 성능 우수)
embeddings_bge = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cpu'},        # 'cuda' for GPU
    encode_kwargs={'normalize_embeddings': True}  # L2 정규화 - 벡터의 각 차원을 벡터의 L2 노름(크기)으로 나누어 단위 벡터로 변환 (벡터 크기 1로 정규화)
)

# BGE-M3 모델로 문서 임베딩
bge_hf_embeddings = embeddings_bge.embed_documents(documents)
print(f"한국어 임베딩 차원: {len(bge_hf_embeddings[0])}")

한국어 임베딩 차원: 1024


#### 📱 경량 모델

In [42]:
# 빠른 처리를 위한 경량 모델
embedding_gte = HuggingFaceEmbeddings(
    model_name="Alibaba-NLP/gte-multilingual-base",
    model_kwargs={'device': 'cpu', 'trust_remote_code': True},  # trust_remote_code 필요 
    encode_kwargs={'normalize_embeddings': True}
)
    
# 경량 모델로 문서 임베딩
alibaba_hf_embeddings = embedding_gte.embed_documents(documents)
print(f"경량 모델 한국어 임베딩 차원: {len(alibaba_hf_embeddings[0])}") 

Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


경량 모델 한국어 임베딩 차원: 768


### 3. Ollama 임베딩 (로컬)

In [45]:
from langchain_ollama import OllamaEmbeddings

# Ollama 서버가 실행 중이어야 함
embeddings_ollama = OllamaEmbeddings(
    model="bge-m3",                    # 사용할 모델
    # base_url="http://localhost:11434"  # Ollama 서버 주소
)

# 로컬 임베딩
local_embeddings = embeddings_ollama.embed_documents(documents)

print(f"로컬 임베딩 벡터 수: {len(local_embeddings)}")
print(f"각 벡터 차원: {len(local_embeddings[0])}")

로컬 임베딩 벡터 수: 5
각 벡터 차원: 1024


### 4. 유사도 계산 및 검색

#### 📏 코사인 유사도

In [46]:
from langchain_community.utils.math import cosine_similarity
import numpy as np

def find_most_similar(query, doc_embeddings, documents, embeddings_model):
    """가장 유사한 문서 찾기"""
    # 쿼리 임베딩
    query_embedding = embeddings_model.embed_query(query)
    
    # 코사인 유사도 계산
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    
    # 가장 유사한 문서 인덱스
    most_similar_idx = np.argmax(similarities)
    
    return {
        "document": documents[most_similar_idx],
        "similarity": similarities[most_similar_idx],
        "index": most_similar_idx
    }

# 쿼리와 문서 임베딩을 사용하여 가장 유사한 문서 찾기 (OpenAI)
query = "딥러닝에 대해 알려주세요"
result = find_most_similar(query, doc_embeddings, documents, embeddings_model)

print(f"쿼리: {query}")
print(f"가장 유사한 문서: {result['document']}")
print(f"유사도 점수: {result['similarity']:.4f}")

100%|██████████| 1/1 [00:01<00:00,  1.30s/it]

쿼리: 딥러닝에 대해 알려주세요
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도 점수: 0.5867


In [47]:
# HuggingFaceEmbeddings를 사용한 유사도 검색 (BGE-M3)
result = find_most_similar(query, bge_hf_embeddings, documents, embeddings_bge)
print(f"쿼리: {query}")
print(f"가장 유사한 문서: {result['document']}")
print(f"유사도 점수: {result['similarity']:.4f}")

쿼리: 딥러닝에 대해 알려주세요
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도 점수: 0.7360


In [48]:
# Alibaba-NLP/gte-multilingual-base 모델로 유사도 검색
result = find_most_similar(query, alibaba_hf_embeddings, documents, embedding_gte)
print(f"쿼리: {query}")
print(f"가장 유사한 문서: {result['document']}")
print(f"유사도 점수: {result['similarity']:.4f}")

쿼리: 딥러닝에 대해 알려주세요
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도 점수: 0.7886


In [49]:
# Ollama 모델로 유사도 검색 (bge-m3)
result = find_most_similar(query, local_embeddings, documents, embeddings_ollama)
print(f"쿼리: {query}")
print(f"가장 유사한 문서: {result['document']}")
print(f"유사도 점수: {result['similarity']:.4f}")

쿼리: 딥러닝에 대해 알려주세요
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도 점수: 0.7350


### 🎯 실습 3: 임베딩 모델 비교

In [50]:
# 다음 질문들에 대해 다른 임베딩 모델들의 검색 성능을 비교해보세요
queries = [
    "기계학습이란 무엇인가요?",
    "이미지 인식 기술에 대해 설명해주세요",
    "언어 모델의 작동 원리는?"
]

# 여기에 코드를 작성하세요


def find_most_similar(query, doc_embeddings, documents, embeddings_model):
    """가장 유사한 문서 찾기"""
    # 쿼리 임베딩
    query_embedding = embeddings_model.embed_query(query)
    
    # 코사인 유사도 계산
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]
    
    # 가장 유사한 문서 인덱스
    most_similar_idx = np.argmax(similarities)
    
    return {
        "document": documents[most_similar_idx],
        "similarity": similarities[most_similar_idx],
        "index": most_similar_idx
    }

# 쿼리와 문서 임베딩을 사용하여 가장 유사한 문서 찾기 (OpenAI)

for query in queries:
    print("query:", query)
    print("embeddings_model")
    result = find_most_similar(query, doc_embeddings, documents, embeddings_model)
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {result['document']}")
    print(f"유사도 점수: {result['similarity']:.4f}")
    print("###########################################################################")


    # HuggingFaceEmbeddings를 사용한 유사도 검색 (BGE-M3)
    print("embeddings_bge")
    result = find_most_similar(query, bge_hf_embeddings, documents, embeddings_bge)
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {result['document']}")
    print(f"유사도 점수: {result['similarity']:.4f}")
    print("###########################################################################")


    # Alibaba-NLP/gte-multilingual-base 모델로 유사도 검색
    print("embedding_gte")
    result = find_most_similar(query, alibaba_hf_embeddings, documents, embedding_gte)
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {result['document']}")
    print(f"유사도 점수: {result['similarity']:.4f}")
    print("###########################################################################")


    # Ollama 모델로 유사도 검색 (bge-m3)
    print("embeddings_ollama")
    result = find_most_similar(query, local_embeddings, documents, embeddings_ollama)
    print(f"쿼리: {query}")
    print(f"가장 유사한 문서: {result['document']}")
    print(f"유사도 점수: {result['similarity']:.4f}")
    print("###########################################################################")




query: 기계학습이란 무엇인가요?
embeddings_model


100%|██████████| 1/1 [00:00<00:00,  3.25it/s]


쿼리: 기계학습이란 무엇인가요?
가장 유사한 문서: 인공지능은 컴퓨터 과학의 한 분야입니다.
유사도 점수: 0.2103
###########################################################################
embeddings_bge
쿼리: 기계학습이란 무엇인가요?
가장 유사한 문서: 머신러닝은 인공지능의 하위 분야입니다.
유사도 점수: 0.5844
###########################################################################
embedding_gte
쿼리: 기계학습이란 무엇인가요?
가장 유사한 문서: 딥러닝은 머신러닝의 한 종류입니다.
유사도 점수: 0.6941
###########################################################################
embeddings_ollama
쿼리: 기계학습이란 무엇인가요?
가장 유사한 문서: 머신러닝은 인공지능의 하위 분야입니다.
유사도 점수: 0.5835
###########################################################################
query: 이미지 인식 기술에 대해 설명해주세요
embeddings_model


100%|██████████| 1/1 [00:00<00:00,  2.07it/s]


쿼리: 이미지 인식 기술에 대해 설명해주세요
가장 유사한 문서: 컴퓨터 비전은 이미지를 분석하는 기술입니다.
유사도 점수: 0.4341
###########################################################################
embeddings_bge
쿼리: 이미지 인식 기술에 대해 설명해주세요
가장 유사한 문서: 컴퓨터 비전은 이미지를 분석하는 기술입니다.
유사도 점수: 0.6503
###########################################################################
embedding_gte
쿼리: 이미지 인식 기술에 대해 설명해주세요
가장 유사한 문서: 컴퓨터 비전은 이미지를 분석하는 기술입니다.
유사도 점수: 0.6549
###########################################################################
embeddings_ollama
쿼리: 이미지 인식 기술에 대해 설명해주세요
가장 유사한 문서: 컴퓨터 비전은 이미지를 분석하는 기술입니다.
유사도 점수: 0.6493
###########################################################################
query: 언어 모델의 작동 원리는?
embeddings_model


100%|██████████| 1/1 [00:00<00:00,  2.82it/s]


쿼리: 언어 모델의 작동 원리는?
가장 유사한 문서: 자연어 처리는 컴퓨터가 인간의 언어를 이해하는 기술입니다.
유사도 점수: 0.3740
###########################################################################
embeddings_bge
쿼리: 언어 모델의 작동 원리는?
가장 유사한 문서: 자연어 처리는 컴퓨터가 인간의 언어를 이해하는 기술입니다.
유사도 점수: 0.5323
###########################################################################
embedding_gte
쿼리: 언어 모델의 작동 원리는?
가장 유사한 문서: 자연어 처리는 컴퓨터가 인간의 언어를 이해하는 기술입니다.
유사도 점수: 0.5514
###########################################################################
embeddings_ollama
쿼리: 언어 모델의 작동 원리는?
가장 유사한 문서: 자연어 처리는 컴퓨터가 인간의 언어를 이해하는 기술입니다.
유사도 점수: 0.5311
###########################################################################


---

## 벡터 저장소 (Vector Store)

### 🎯 벡터 저장소란?
임베딩된 벡터를 효율적으로 저장하고 유사도 기반 검색을 수행하는 특수 데이터베이스

### 📊 벡터 저장소 비교
| 종류 | 장점 | 단점 | 사용 사례 |
|------|------|------|----------|
| Chroma | 설치 간단, 로컬 친화적 | 대용량 처리 한계 | 개발, 프로토타입 |
| FAISS | 매우 빠름, 확장성 우수 | 설정 복잡 | 대용량 검색 |
| Pinecone | 완전 관리형, 고성능 | 유료, 클라우드 의존 | 프로덕션 |
| Weaviate | GraphQL 지원, 하이브리드 검색 | 학습 곡선 | 복합 검색 |

### 🚀 Chroma 설치 및 설정
```bash
pip install langchain-chroma
```

#### 📚 기본 사용법

In [52]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 준비
pdf_loader = PyPDFLoader('./data/labor_law.pdf', mode='single')
pdf_docs = pdf_loader.load()

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=500,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(pdf_docs)
print(f"생성된 청크 수: {len(chunks)}")

# 임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 저장소 생성
chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="labor_law",
    persist_directory="./local_chroma_db",
    collection_metadata={"hnsw:space": "cosine"}  # 유사도 메트릭
)

print(f"저장된 문서 수: {chroma_db._collection.count()}")

생성된 청크 수: 89
저장된 문서 수: 89


#### 💾 벡터 저장소 로드

In [53]:
# 기존 벡터 저장소 로드
chroma_db = Chroma(
    collection_name="labor_law",
    embedding_function=embeddings,
    persist_directory="./local_chroma_db"
)

print(f"로드된 문서 수: {chroma_db._collection.count()}")

로드된 문서 수: 89


#### 🔍 기본 검색 기능

In [54]:
# 1. 유사도 검색
query = "탄력 근로에 대해 설명해주세요"
similar_docs = chroma_db.similarity_search(
    query=query,
    k=5,  # 상위 5개 결과
    filter={"source": "./data/labor_law.pdf"}  # 메타데이터 필터
)

print(f"검색 결과 수: {len(similar_docs)}")
for i, doc in enumerate(similar_docs):
    print(f"결과 {i+1}: {doc.page_content[:100]}...")
    print("-" * 40)

검색 결과 수: 5
결과 1: ② 1일의 근로시간은 휴게시간을 제외하고 8시간을 초과할 수 없다.
③ 제1항 및 제2항에 따라 근로시간을 산정하는 경우 작업을 위하여 근로자가 사용자의 지휘ㆍ감독 아래에 있는 대...
----------------------------------------
결과 2: 근로기준법
[제목개정 2021. 1. 5.]
 
제51조의2(3개월을 초과하는 탄력적 근로시간제) ① 사용자는 근로자대표와의 서면 합의에 따라 다음 각 호의 사항을
정하면 3개월을...
----------------------------------------
결과 3: 다. 다만, 조사와 관련된 내용을 사용자에게 보고하거나 관계 기관의 요청에 따라 필요한 정보를 제공하는 경우는
제외한다.<신설 2021. 4. 13.>
[본조신설 2019. 1. ...
----------------------------------------
결과 4: 자의 근로시간을 기준으로 산정한 비율에 따라 결정되어야 한다.
② 제1항에 따라 근로조건을 결정할 때에 기준이 되는 사항이나 그 밖에 필요한 사항은 대통령령으로 정한다.
③ 4주 ...
----------------------------------------
결과 5: 간의 근로시간이 제50조제1항의 근로시간을 초과하지 아니하는 범위에서 특정한 주에 제50조제1항의 근로시간을,
특정한 날에 제50조제2항의 근로시간을 초과하여 근로하게 할 수 있다...
----------------------------------------


In [55]:
# 2. 점수와 함께 검색 (유사도 점수 포함)
docs_with_scores = chroma_db.similarity_search_with_score(query, k=3)

for doc, score in docs_with_scores:
    print(f"점수: {score:.4f}")
    print(f"내용: {doc.page_content[:100]}...")
    print("-" * 50)

점수: 0.6894
내용: ② 1일의 근로시간은 휴게시간을 제외하고 8시간을 초과할 수 없다.
③ 제1항 및 제2항에 따라 근로시간을 산정하는 경우 작업을 위하여 근로자가 사용자의 지휘ㆍ감독 아래에 있는 대...
--------------------------------------------------
점수: 0.7102
내용: 근로기준법
[제목개정 2021. 1. 5.]
 
제51조의2(3개월을 초과하는 탄력적 근로시간제) ① 사용자는 근로자대표와의 서면 합의에 따라 다음 각 호의 사항을
정하면 3개월을...
--------------------------------------------------
점수: 0.7346
내용: 다. 다만, 조사와 관련된 내용을 사용자에게 보고하거나 관계 기관의 요청에 따라 필요한 정보를 제공하는 경우는
제외한다.<신설 2021. 4. 13.>
[본조신설 2019. 1. ...
--------------------------------------------------


#### 🎛️ 메타데이터 필터링

In [56]:
# 복합 필터 조건
filter_criteria = {
    "$and": [
        {"source": {"$eq": "./data/labor_law.pdf"}},
        {"page": {"$gte": 10}}  # 10페이지 이상
    ]
}

filtered_results = chroma_db.similarity_search(
    query=query,
    k=5,
    filter=filter_criteria, 
)

print(f"필터링된 검색 결과 수: {len(filtered_results)}")
for i, doc in enumerate(filtered_results):
    print(f"결과 {i+1}: {doc.page_content[:100]}...")
    print("-" * 40)

필터링된 검색 결과 수: 0


#### 🔄 문서 업데이트

In [57]:
# 새 문서 추가
from langchain_core.documents import Document

new_docs = [Document(page_content="새로운 내용", metadata={"source": "new"})]
chroma_db.add_documents(new_docs, ids=["new_doc_1"])

['new_doc_1']

In [58]:
# 문서 삭제 (ID 기반)
chroma_db.delete(ids=["new_doc_1"])

In [ ]:
# 전체 컬렉션 삭제
# chroma_db.delete_collection()

### 🎯 실습 4: 벡터 저장소 구축

In [75]:


# 문서 삭제 (ID 기반)
# chroma_db.delete(ids=["web_docs1"])
# chroma_db.delete_collection()

# 기존 벡터 저장소 로드 확인
chroma_db = Chroma(
    collection_name="web_docs1",
    embedding_function=embeddings,
    persist_directory="./web_chroma_db"
)
print(f"로드된 문서 수: {chroma_db._collection.count()}")


로드된 문서 수: 12


In [74]:
# 다음 단계로 나만의 벡터 저장소를 구축해보세요:
# 1. 웹에서 문서 로드
# 2. 적절한 크기로 분할
# 3. 임베딩 및 저장
# 4. 검색 테스트

# 여기에 코드를 작성하세요


# 1. 웹 문서 로드
urls = [
    "https://python.langchain.com/docs/tutorials/",
]
web_loader = WebBaseLoader(web_paths=urls)
web_docs = web_loader.load()
print(f'웹 문서 개수: {len(web_docs)}')


# 2. 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",
    chunk_size=300,
    chunk_overlap=100
)
chunks = text_splitter.split_documents(web_docs)
print(f"생성된 청크 수: {len(chunks)}")



# 3. 임베딩 모델 
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")


# 벡터에 저장할때, chunk를 embedding 해서 저장하는 과정이 같이 진행됨.
# bge_hf_embeddings = embeddings_bge.embed_documents(chunks)


# Chroma 벡터 저장소 생성
chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    collection_name="web_docs1",
    persist_directory="./web_chroma_db",
    collection_metadata={"hnsw:space": "cosine"}  # 유사도 메트릭
)
print(f"저장된 문서 수: {chroma_db._collection.count()}")


# 기존 벡터 저장소 로드 확인
chroma_db = Chroma(
    collection_name="web_docs1",
    embedding_function=embeddings,
    persist_directory="./web_chroma_db"
)
print(f"로드된 문서 수: {chroma_db._collection.count()}")


# 4. 검색 테스트
query = "LangChain 튜토리얼에 대해 알려주세요"
# similar_docs = chroma_db.similarity_search(
#     query=query,
#     k=5  # 상위 5개 결과
# )
# print(f"검색 결과 수: {len(similar_docs)}")
# for i, doc in enumerate(similar_docs):
#     print(f"결과 {i+1}: {doc.page_content[:100]}...")
#     print("-" * 40)


docs_with_scores = chroma_db.similarity_search_with_score(query, k=5)

for doc, score in docs_with_scores:
    print(f"점수: {score:.4f}")
    print(f"내용: {doc.page_content[:100]}...")
    print("-" * 50)

# # HuggingFaceEmbeddings를 사용한 유사도 검색 (BGE-M3)
# result = find_most_similar(query, bge_hf_embeddings, documents, embeddings_bge)
# print(f"쿼리: {query}")
# print(f"가장 유사한 문서: {result['document']}")
# print(f"유사도 점수: {result['similarity']:.4f}")


웹 문서 개수: 1
생성된 청크 수: 12
저장된 문서 수: 12
로드된 문서 수: 12
점수: 1.0141
내용: Tutorials | 🦜️🔗 LangChain...
--------------------------------------------------
점수: 1.1930
내용: prompting with tool callingHow to add a human-in-the-loop for toolsHow to bind model-specific toolsH...
--------------------------------------------------
점수: 1.2091
내용: Skip to main contentOur new LangChain Academy Course Deep Research with LangGraph is now live! Enrol...
--------------------------------------------------
점수: 1.2130
내용: to do "self-querying" retrievalHow to split text based on semantic similarityHow to chain runnablesH...
--------------------------------------------------
점수: 1.2134
내용: LangSmith​
LangSmith allows you to closely trace, monitor and evaluate your LLM application.
It seam...
--------------------------------------------------


---

## 검색기 (Retriever)

### 🎯 Retriever란?
벡터 저장소를 기반으로 사용자 질의에 가장 관련성 높은 문서를 검색하는 인터페이스입니다.

### 📊 검색 전략 비교
| 전략 | 설명 | 장점 | 단점 | 사용 사례 |
|------|------|------|------|----------|
| similarity | 단순 유사도 검색 | 빠름, 직관적 | 다양성 부족 | 일반적인 검색 |
| similarity_score_threshold | 임계값 기반 검색 | 품질 보장 | 결과 수 불안정 | 고품질 결과 필요 |
| mmr | 최대 한계 관련성 | 다양성 우수 | 느림 | 포괄적 정보 필요 |

### 1. 기본 유사도 검색

#### 🔍 Top-K 검색

In [76]:
# 벡터 저장소를 Retriever로 변환
retriever = chroma_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # 상위 5개 결과
)

# 검색 실행
query = "탄력 근로에 대해 설명해주세요"
retrieved_docs = retriever.invoke(query)

print(f"검색된 문서 수: {len(retrieved_docs)}")
for i, doc in enumerate(retrieved_docs):
    print(f"문서 {i+1}:")
    print(f"내용: {doc.page_content[:200]}...")
    print(f"출처: {doc.metadata.get('source', 'Unknown')}")
    print("-" * 50)

검색된 문서 수: 5
문서 1:
내용: Tutorials | 🦜️🔗 LangChain...
출처: https://python.langchain.com/docs/tutorials/
--------------------------------------------------
문서 2:
내용: LangSmith​
LangSmith allows you to closely trace, monitor and evaluate your LLM application.
It seamlessly integrates with LangChain, and you can use it to inspect and debug individual steps of your c...
출처: https://python.langchain.com/docs/tutorials/
--------------------------------------------------
문서 3:
내용: from LLMChainMigrating from LLMMathChainMigrating from LLMRouterChainMigrating from MapReduceDocumentsChainMigrating from MapRerankDocumentsChainMigrating from MultiPromptChainMigrating from RefineDoc...
출처: https://python.langchain.com/docs/tutorials/
--------------------------------------------------
문서 4:
내용: to do "self-querying" retrievalHow to split text based on semantic similarityHow to chain runnablesHow to save and load LangChain objectsHow to split text by tokensHow to split HTMLHow to do question ...
출처: htt

### 2. 임계값 기반 검색

#### 📏 점수 임계값 설정

In [77]:
# 유사도 점수 임계값 기반 검색
threshold_retriever = chroma_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold": 0.3,  # 0.3 이상의 유사도만
        "k": 10                  # 최대 10개까지
    }
)

retrieved_docs = threshold_retriever.invoke(query)

# 실제 유사도 점수 확인
from langchain_community.utils.math import cosine_similarity

for i, doc in enumerate(retrieved_docs):
    # 실제 유사도 계산
    doc_embedding = embeddings.embed_query(doc.page_content)
    query_embedding = embeddings.embed_query(query)
    similarity = cosine_similarity([query_embedding], [doc_embedding])[0][0]
    
    print(f"문서 {i+1} (유사도: {similarity:.4f}):")
    print(f"{doc.page_content[:100]}...")
    print()

c:\Users\ys\ktds-llm\.venv\Lib\site-packages\langchain_core\vectorstores\base.py:1083: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='9d2e73e2-be6e-484e-a6a0-af8b392c3b12', metadata={'language': 'en', 'title': 'Tutorials | 🦜️🔗 LangChain', 'source': 'https://python.langchain.com/docs/tutorials/', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.'}, page_content='Tutorials | 🦜️🔗 LangChain'), -0.14500621518745982), (Document(id='e64f05a3-877c-4e74-9f4e-e16fc14f5081', metadata={'language': 'en', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | 🦜️🔗 LangChain'}, page_content='LangSmith\u200b\nLangSmith allows you to closely trace, monitor and evaluate your LLM application.\nIt seamless

### 3. MMR (Maximal Marginal Relevance) 검색

#### 🎯 다양성을 고려한 검색


In [78]:
# MMR 검색 - 관련성과 다양성의 균형
mmr_retriever = chroma_db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 5,                # 최종 반환할 문서 수
        "fetch_k": 20,         # 초기 후보 문서 수
        "lambda_mult": 0.5     # 관련성 vs 다양성 (0=최대 다양성, 1=최대 관련성)
    }
)

mmr_docs = mmr_retriever.invoke(query)

print("MMR 검색 결과:")
for i, doc in enumerate(mmr_docs):
    print(f"문서 {i+1}: {doc.page_content[:150]}...")
    print()

MMR 검색 결과:
문서 1: Tutorials | 🦜️🔗 LangChain...

문서 2: LangSmith​
LangSmith allows you to closely trace, monitor and evaluate your LLM application.
It seamlessly integrates with LangChain, and you can use ...

문서 3: from LLMChainMigrating from LLMMathChainMigrating from LLMRouterChainMigrating from MapReduceDocumentsChainMigrating from MapRerankDocumentsChainMigra...

문서 4: one step to the nextHow to compose prompts togetherHow to handle multiple retrievers when doing query analysisHow to add values to a chain's stateHow ...

문서 5: New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.
Get started​
Fa...



#### 🔧 lambda_mult 파라미터 실험


In [79]:
# 다양한 lambda_mult 값으로 실험
lambda_values = [0.0, 0.25, 0.5, 0.75, 1.0]

for lambda_val in lambda_values:
    retriever = chroma_db.as_retriever(
        search_type="mmr",
        search_kwargs={
            "k": 3,
            "fetch_k": 10,
            "lambda_mult": lambda_val
        }
    )
    
    docs = retriever.invoke(query)
    print(f"\nLambda {lambda_val} 결과:")
    print("-" * 40)
    for i, doc in enumerate(docs):
        print(f"  {i+1}. {doc.page_content[:100]}...")
    
    print("=" * 40)


Lambda 0.0 결과:
----------------------------------------
  1. Tutorials | 🦜️🔗 LangChain...
  2. LangSmith​
LangSmith allows you to closely trace, monitor and evaluate your LLM application.
It seam...
  3. one step to the nextHow to compose prompts togetherHow to handle multiple retrievers when doing quer...

Lambda 0.25 결과:
----------------------------------------
  1. Tutorials | 🦜️🔗 LangChain...
  2. LangSmith​
LangSmith allows you to closely trace, monitor and evaluate your LLM application.
It seam...
  3. one step to the nextHow to compose prompts togetherHow to handle multiple retrievers when doing quer...

Lambda 0.5 결과:
----------------------------------------
  1. Tutorials | 🦜️🔗 LangChain...
  2. LangSmith​
LangSmith allows you to closely trace, monitor and evaluate your LLM application.
It seam...
  3. one step to the nextHow to compose prompts togetherHow to handle multiple retrievers when doing quer...

Lambda 0.75 결과:
----------------------------------------
  1. Tutorials

#### 🎛️ 메타데이터 필터링

In [82]:
# 메타데이터 기반 필터링 retriever
filtered_retriever = chroma_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 5,
        "filter": {
            "source": "./data/labor_law.pdf"
        }
    }
)

filtered_results = filtered_retriever.invoke(query)

print("메타데이터 기반 필터링 결과:")
for i, doc in enumerate(filtered_results):
    print(f"문서 {i+1}: {doc.page_content[:150]}...")
    print()

메타데이터 기반 필터링 결과:


#### 🔄 동적 검색 파라미터

In [83]:
class DynamicRetriever:
    def __init__(self, vectorstore, embeddings):
        self.vectorstore = vectorstore
        self.embeddings = embeddings
    
    def retrieve(self, query, search_type="auto", k=5):
        """쿼리 특성에 따라 동적으로 검색 전략 선택"""
        
        # 쿼리 복잡도 분석
        query_length = len(query.split())
        
        if query_length <= 3:
            # 짧은 쿼리: 높은 임계값
            search_type = "similarity_score_threshold"
            search_kwargs = {"score_threshold": 0.25, "k": k}
        elif query_length > 10:
            # 긴 쿼리: MMR로 다양성 확보
            search_type = "mmr"
            search_kwargs = {"k": k, "fetch_k": k*3, "lambda_mult": 0.3}
        else:
            # 중간 길이: 기본 유사도 검색
            search_type = "similarity"
            search_kwargs = {"k": k}
        
        retriever = self.vectorstore.as_retriever(
            search_type=search_type,
            search_kwargs=search_kwargs
        )
        
        return retriever.invoke(query)

# 사용 예시
dynamic_retriever = DynamicRetriever(chroma_db, embeddings)

queries = [
    "탄력근로",  # 짧은 쿼리
    "탄력 근로에 대해 설명해주세요",  # 중간 쿼리
    "탄력 근로 제도의 장점과 단점, 그리고 실제 적용 사례를 포함하여 자세히 설명해주세요"  # 긴 쿼리
]

for query in queries:
    print(f"\n쿼리: {query}")
    print(f"길이: {len(query.split())} 단어")
    docs = dynamic_retriever.retrieve(query)
    print(f"검색 결과: {len(docs)}개 문서")
    print("-" * 40)


쿼리: 탄력근로
길이: 1 단어


c:\Users\ys\ktds-llm\.venv\Lib\site-packages\langchain_core\vectorstores\base.py:1083: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='e64f05a3-877c-4e74-9f4e-e16fc14f5081', metadata={'language': 'en', 'title': 'Tutorials | 🦜️🔗 LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'source': 'https://python.langchain.com/docs/tutorials/'}, page_content='LangSmith\u200b\nLangSmith allows you to closely trace, monitor and evaluate your LLM application.\nIt seamlessly integrates with LangChain, and you can use it to inspect and debug individual steps of your chains as you build.\nLangSmith documentation is hosted on a separate site.\nYou can peruse LangSmith tutorials here.\nEvaluation\u200b\nLangSmith helps you evaluate the performance of your LLM applications. The tutorial below is a great way to get started:\n\nEvaluate your LLM application\nEdit this 

검색 결과: 0개 문서
----------------------------------------

쿼리: 탄력 근로에 대해 설명해주세요
길이: 4 단어
검색 결과: 5개 문서
----------------------------------------

쿼리: 탄력 근로 제도의 장점과 단점, 그리고 실제 적용 사례를 포함하여 자세히 설명해주세요
길이: 12 단어
검색 결과: 5개 문서
----------------------------------------


### 🎯 실습 5: 검색 전략 비교

In [85]:
# 같은 질문에 대해 다른 검색 전략들의 결과를 비교해보세요
test_query = "근로시간 단축에 대한 규정은 무엇인가요?"

strategies = {
    "similarity": {"k": 5},
    "similarity_score_threshold": {"score_threshold": 0.3, "k": 10},
    "mmr": {"k": 5, "fetch_k": 15, "lambda_mult": 0.5}
}

# 여기에 코드를 작성하세요

---

## RAG 체인 구현

### 🎯 RAG 체인이란?
검색(Retrieval)과 생성(Generation)을 연결하여 외부 지식을 기반으로 답변을 생성하는 파이프라인

### 🔄 RAG 워크플로우
`
사용자 질문 → 관련 문서 검색 → 컨텍스트 구성 → LLM 프롬프트 → 답변 생성
`

### 1. 프롬프트 템플릿 설계

#### 📝 기본 RAG 프롬프트

In [86]:
from langchain_core.prompts import ChatPromptTemplate

# 기본 RAG 프롬프트 템플릿
basic_template = """주어진 컨텍스트를 기반으로 질문에 답변하세요.

컨텍스트:
{context}

질문: {question}

답변:"""

basic_prompt = ChatPromptTemplate.from_template(basic_template)

#### 🎨 고급 RAG 프롬프트

In [87]:
advanced_template = """당신은 전문적인 문서 분석 AI입니다. 주어진 컨텍스트를 바탕으로 정확하고 유용한 답변을 제공하세요.

## 답변 지침
- 컨텍스트에 있는 정보만을 사용하여 답변하세요
- 확실하지 않은 정보는 "명확하지 않습니다"라고 명시하세요
- 답변은 논리적이고 구조화된 형태로 제공하세요
- 가능한 경우 구체적인 예시나 수치를 포함하세요
- 답변에 참조한 문서의 출처가 있다면 포함하세요 (문서명, 페이지, URL 등)

## 컨텍스트
{context}

## 질문
{question}

## 답변 형식
**핵심 답변:** (질문에 대한 직접적인 답변)

**세부 설명:** (추가적인 설명이나 배경 정보)

**관련 정보:** (컨텍스트에서 발견된 연관 정보)

**답변:**"""

advanced_prompt = ChatPromptTemplate.from_template(advanced_template)

#### 🌟 도메인별 특화 프롬프트

In [88]:
legal_template = """당신은 법률 문서 전문 AI입니다. 법률 조항을 정확히 해석하고 설명해주세요.

## 법률 해석 원칙
- 조문의 정확한 인용을 포함하세요
- 법적 용어는 일반인이 이해할 수 있도록 설명하세요
- 예외 조항이나 단서가 있다면 반드시 언급하세요
- 관련 법령이나 시행령도 함께 언급하세요

## 관련 법률 조항
{context}

## 법률 질의
{question}

## 법률 답변
**해당 조항:** (관련 법률 조항 인용)

**조항 해석:** (조항의 의미와 적용 범위)

**주의사항:** (예외 조항이나 제한 사항)

**답변:**"""

legal_prompt = ChatPromptTemplate.from_template(legal_template)

### 2. LLM 설정

In [89]:
from langchain_openai import ChatOpenAI

# LLM 설정
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.1,                   # 일관성 있는 답변
    max_completion_tokens=1000,        # 답변 길이 제한
    top_p=0.9,              # 다양성 제어
    # frequency_penalty=0.1,  # 반복 방지
    # presence_penalty=0.1    # 새로운 단어 장려
)

### 3. RAG 체인 구성

#### 🔗 기본 LCEL 체인

In [90]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    """문서 리스트를 문자열로 포맷"""
    return "\n\n".join([
        f"{doc.page_content}" for doc in docs
    ])

# 기본 RAG 체인
basic_rag_chain = (
    RunnableParallel({
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    })
    | basic_prompt
    | llm
    | StrOutputParser()
)

# 테스트
query = "탄력 근로에 대해 설명해주세요"
result = basic_rag_chain.invoke(query)
print(result)

탄력 근로제는 근로자가 일정 기간 동안 근로 시간을 자유롭게 조정할 수 있도록 하는 제도입니다. 예를 들어, 주당 법정 근로 시간 내에서 하루에 일하는 시간을 늘리거나 줄이는 대신, 다른 날에 근로 시간을 조정하여 전체 근로 시간을 맞추는 방식입니다. 이를 통해 근로자는 개인의 상황이나 업무량에 맞게 근무 시간을 유연하게 운영할 수 있고, 기업은 업무 효율성을 높일 수 있습니다. 다만, 탄력 근로제를 도입할 때는 근로기준법 등 관련 법규를 준수해야 하며, 근로자와의 합의가 필요합니다.


#### 🎯 고급 RAG 체인

In [91]:
def format_docs_with_metadata(docs):
    """메타데이터를 포함한 문서 포맷팅"""
    formatted_docs = []
    for i, doc in enumerate(docs):
        source = doc.metadata.get('source', 'Unknown')
        page = doc.metadata.get('page', 'N/A')
        
        formatted_doc = f"""
출처: {source}
페이지: {page}
내용: {doc.page_content}
---
"""
        formatted_docs.append(formatted_doc)
    
    return "\n".join(formatted_docs)


# 고급 RAG 체인
advanced_rag_chain = (
    RunnableParallel({
        "context": retriever | format_docs_with_metadata,
        "question": RunnablePassthrough()
    })
    | advanced_prompt
    | llm
    | StrOutputParser()
)

# 테스트
query = "탄력 근로에 대해 설명해주세요"
result = advanced_rag_chain.invoke(query)
print(result)

**핵심 답변:**  
컨텍스트 내에는 탄력 근로에 대한 설명이나 정의가 포함되어 있지 않아, 탄력 근로에 대해 명확하게 설명할 수 없습니다.

**세부 설명:**  
탄력 근로제는 일반적으로 근로시간을 일정 기간 단위로 유연하게 조정하여 근로자와 사용자가 근로시간을 탄력적으로 운영할 수 있도록 하는 제도입니다. 하지만 주어진 컨텍스트에서는 이와 관련된 정보가 전혀 제공되지 않았습니다.

**관련 정보:**  
컨텍스트는 주로 LangChain과 LangSmith 관련 튜토리얼, 평가, 마이그레이션, 메모리 관리 등에 관한 내용으로 구성되어 있으며, 탄력 근로와 관련된 내용은 포함되어 있지 않습니다.

**답변:**  
주어진 컨텍스트에는 탄력 근로에 관한 정보가 포함되어 있지 않아, 이에 대해 설명드리기 어렵습니다. 추가적인 자료나 문서가 제공되면 그에 맞춰 답변을 드릴 수 있습니다.


### 🎯 실습 6: 완전한 RAG 시스템 구축

In [ ]:
# 다음 요구사항을 만족하는 RAG 시스템을 구축해보세요:
# 1. 여러 문서 형식 지원 (PDF, 웹, 텍스트)
# 2. 동적 검색 전략 선택
# 3. 도메인별 프롬프트 템플릿
# 4. 응답 품질 평가

class ComprehensiveRAGSystem:
    def __init__(self):
        pass
    
    def load_documents(self, sources):
        """다양한 소스에서 문서 로드"""
        # 여기에 코드를 작성하세요
        pass
    
    def setup_vector_store(self, documents):
        """벡터 저장소 구성"""
        # 여기에 코드를 작성하세요
        pass
    
    def query(self, question, domain="general"):
        """도메인별 질의응답"""
        # 여기에 코드를 작성하세요
        pass

# 여기에 구현 코드를 작성하세요